<a href="https://colab.research.google.com/github/sofaemha/archive/blob/main/2025-10/spk-ahp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [2]:

def normalize_matrix(matrix):
    """Normalize the matrix by dividing each element by the column sum."""
    col_sums = np.sum(matrix, axis=0)
    return matrix / col_sums

def calculate_priority_vector(normalized_matrix):
    """Calculate the priority vector by averaging the rows."""
    return np.mean(normalized_matrix, axis=1)

def calculate_lambda_max(matrix, priority_vector):
    """Calculate the maximum eigenvalue (lambda_max)."""
    return np.mean(np.dot(matrix, priority_vector) / priority_vector)

def calculate_consistency_index(lambda_max, n):
    """Calculate the Consistency Index (CI)."""
    return (lambda_max - n) / (n - 1)

def get_random_index(n):
    """Get the Random Index (RI) for a given matrix size n."""
    ri_values = {
        1: 0.00, 2: 0.00, 3: 0.58, 4: 0.90, 5: 1.12,
        6: 1.24, 7: 1.32, 8: 1.41, 9: 1.45, 10: 1.49
    }
    return ri_values.get(n, 1.49)  # Default to 1.49 for n > 10

def calculate_consistency_ratio(ci, ri):
    """Calculate the Consistency Ratio (CR)."""
    return ci / ri

def ahp(pairwise_matrix):
    """Perform AHP calculation on a pairwise comparison matrix."""
    n = pairwise_matrix.shape[0]

    # Normalize the matrix
    normalized = normalize_matrix(pairwise_matrix)

    # Calculate priority vector (weights)
    weights = calculate_priority_vector(normalized)

    # Calculate lambda_max
    lambda_max = calculate_lambda_max(pairwise_matrix, weights)

    # Calculate CI
    ci = calculate_consistency_index(lambda_max, n)

    # Get RI
    ri = get_random_index(n)

    # Calculate CR
    cr = calculate_consistency_ratio(ci, ri)

    return weights, cr

In [4]:

def main():
    print("AHP (Analytical Hierarchy Process) Calculator")
    print("=============================================")

    # Input number of criteria
    num_criteria = int(input("Enter the number of criteria: "))

    # Initialize pairwise comparison matrix for criteria
    criteria_matrix = np.zeros((num_criteria, num_criteria))

    # Fill the matrix
    print("\nEnter the pairwise comparison values for criteria (use Saaty scale: 1-9, and reciprocals)")
    for i in range(num_criteria):
        for j in range(i + 1, num_criteria):
            value = float(input(f"Compare criterion {i+1} to {j+1}: "))
            criteria_matrix[i, j] = value
            criteria_matrix[j, i] = 1 / value
        criteria_matrix[i, i] = 1.0  # Diagonal is 1

    print("\nCriteria Pairwise Matrix:")
    print(criteria_matrix)

    # Calculate criteria weights
    criteria_weights, criteria_cr = ahp(criteria_matrix)

    print("\nCriteria Weights:")
    for i, w in enumerate(criteria_weights, 1):
        print(f"Criterion {i}: {w:.4f}")

    print(f"Consistency Ratio (CR): {criteria_cr:.4f}")
    if criteria_cr > 0.1:
        print("Warning: CR > 0.1, the matrix may be inconsistent.")

    # Input number of alternatives
    num_alternatives = int(input("\nEnter the number of alternatives: "))

    # For each criterion, create a pairwise matrix for alternatives
    alternative_matrices = []
    alternative_weights = []

    for c in range(num_criteria):
        print(f"\nPairwise comparisons for alternatives under criterion {c+1}")
        alt_matrix = np.zeros((num_alternatives, num_alternatives))
        for i in range(num_alternatives):
            for j in range(i + 1, num_alternatives):
                value = float(input(f"Compare alternative {i+1} to {j+1} under criterion {c+1}: "))
                alt_matrix[i, j] = value
                alt_matrix[j, i] = 1 / value
            alt_matrix[i, i] = 1.0

        print(f"\nAlternative Matrix for Criterion {c+1}:")
        print(alt_matrix)

        alt_weights, alt_cr = ahp(alt_matrix)
        alternative_matrices.append(alt_matrix)
        alternative_weights.append(alt_weights)

        print(f"\nAlternative Weights for Criterion {c+1}:")
        for i, w in enumerate(alt_weights, 1):
            print(f"Alternative {i}: {w:.4f}")
        print(f"Consistency Ratio (CR): {alt_cr:.4f}")
        if alt_cr > 0.1:
            print("Warning: CR > 0.1, the matrix may be inconsistent.")

    # Calculate overall scores
    overall_scores = np.zeros(num_alternatives)
    for a in range(num_alternatives):
        for c in range(num_criteria):
            overall_scores[a] += criteria_weights[c] * alternative_weights[c][a]

    print("\nOverall Scores:")
    for i, score in enumerate(overall_scores, 1):
        print(f"Alternative {i}: {score:.4f}")

    # Rank the alternatives
    ranked = np.argsort(overall_scores)[::-1]
    print("\nRanking:")
    for rank, idx in enumerate(ranked, 1):
        print(f"Rank {rank}: Alternative {idx+1} (Score: {overall_scores[idx]:.4f})")

In [5]:
if __name__ == "__main__":
    main()

AHP (Analytical Hierarchy Process) Calculator
Enter the number of criteria: 5

Enter the pairwise comparison values for criteria (use Saaty scale: 1-9, and reciprocals)
Compare criterion 1 to 2: 3
Compare criterion 1 to 3: 5
Compare criterion 1 to 4: 7
Compare criterion 1 to 5: 9
Compare criterion 2 to 3: 3
Compare criterion 2 to 4: 5
Compare criterion 2 to 5: 7
Compare criterion 3 to 4: 3
Compare criterion 3 to 5: 5
Compare criterion 4 to 5: 3

Criteria Pairwise Matrix:
[[1.         3.         5.         7.         9.        ]
 [0.33333333 1.         3.         5.         7.        ]
 [0.2        0.33333333 1.         3.         5.        ]
 [0.14285714 0.2        0.33333333 1.         3.        ]
 [0.11111111 0.14285714 0.2        0.33333333 1.        ]]

Criteria Weights:
Criterion 1: 0.5028
Criterion 2: 0.2602
Criterion 3: 0.1344
Criterion 4: 0.0678
Criterion 5: 0.0348
Consistency Ratio (CR): 0.0542

Enter the number of alternatives: 3

Pairwise comparisons for alternatives under c